# Titanic : Machine Learning from Disaster
Implementation using **some_fancy_feature_selection_here** feature selection and **some_fancy_classifier_here** classifier

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split, KFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.decomposition import PCA

In [2]:
data_path = "data/"
data_train = pd.read_csv(data_path + "train.csv", delimiter=',')

cls = data_train["Survived"].values

In [3]:
data_train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [20]:
pd.get_dummies(data_train["Pclass"]).info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 3 columns):
1    891 non-null uint8
2    891 non-null uint8
3    891 non-null uint8
dtypes: uint8(3)
memory usage: 2.7 KB


In [18]:
pd.get_dummies(map(lambda x: x.split(',')[1].split('.')[0].strip(), data_train["Name"])).info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 17 columns):
Capt            891 non-null uint8
Col             891 non-null uint8
Don             891 non-null uint8
Dr              891 non-null uint8
Jonkheer        891 non-null uint8
Lady            891 non-null uint8
Major           891 non-null uint8
Master          891 non-null uint8
Miss            891 non-null uint8
Mlle            891 non-null uint8
Mme             891 non-null uint8
Mr              891 non-null uint8
Mrs             891 non-null uint8
Ms              891 non-null uint8
Rev             891 non-null uint8
Sir             891 non-null uint8
the Countess    891 non-null uint8
dtypes: uint8(17)
memory usage: 14.9 KB


In [21]:
pd.get_dummies(data_train["Embarked"]).info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 3 columns):
C    891 non-null uint8
Q    891 non-null uint8
S    891 non-null uint8
dtypes: uint8(3)
memory usage: 2.7 KB


In [ ]:
data_train["Age"] = data_train["Age"].fillna(data_train["Age"].mean())
data_train["Name"] = pd.factorize(map(lambda x: x.split(' ')[1], data_train["Name"]))[0]
data_train["Sex"] = pd.factorize(data_train["Sex"])[0]
data_train["Cabin"] = map(lambda x: x+1, pd.factorize(data_train["Cabin"])[0])
data_train["Embarked"] = pd.factorize(data_train["Embarked"])[0]

data_train_drop = data_train.drop(["PassengerId" ,"Survived", "Ticket", "Parch", "Cabin", "Embarked"], axis=1)

ftr = data_train_drop.values

In [ ]:
def normFeature(x):
    x /= np.linalg.norm(x)
    x = x.tolist()
    return x
ftr = map(lambda x: normFeature(x), ftr)
ftr = np.array(ftr)

In [ ]:
pca = PCA(n_components=2)
ftr_for_plot = pca.fit_transform(ftr)

plt.figure(0)
for i in range(len(ftr)):
    if cls[i] == 0:
        line = plt.plot(ftr_for_plot[i][1], ftr_for_plot[i][0], "ro")
        plt.setp(line, color="r", linewidth=2.0)

plt.figure(1)
for i in range(len(ftr)):
    if cls[i] == 1:
        line = plt.plot(ftr_for_plot[i][1], ftr_for_plot[i][0], "ro")
        plt.setp(line, color="b", linewidth=2.0)

In [ ]:
# x_train, x_test, y_train, y_test = train_test_split(ftr, cls, test_size=0.3)
from sklearn.neighbors import KNeighborsClassifier
accuracy = []
kfold = KFold(n_splits=10)
for train_index, test_index in kfold.split(ftr):
    x_train, x_test = ftr[train_index], ftr[test_index]
    y_train, y_test = cls[train_index], cls[test_index]

    clf = RandomForestClassifier(n_estimators=100, oob_score=True).fit(x_train, y_train)
#     clf = KNeighborsClassifier(n_neighbors=7).fit(x_train, y_train)
    acc = clf.score(x_test, y_test)
    print acc
    accuracy.append(acc)

In [ ]:
print "Average accuracy: " + str(np.mean(accuracy))

In [ ]:
data_train_drop.head()

In [ ]:
print clf.feature_importances_